# practice

### torch

In [9]:
import sys
sys.path.insert(0, '/media/allen/mass/deep-learning-works/')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist, pdist
import importlib

In [47]:
import numpy
from PIL import Image, ImageDraw

# polygon = [(1,1),(2,10),(10,5)]
width = 15
height = 20



In [57]:
x1, y1 = (1,1)
x2, y2 = (5,4)
x, y = abs(x2-x1), abs(y2-y1)
angle = np.arctan(y/x)
x_offset = 2 * np.tan(np.pi/2 - angle)
polygon = np.array([[x1-x_offset, y1-2], [x1+x_offset, y1+2],
                    [x2-x_offset, y2-2], [x2+x_offset, y2+2]]).astype(int)
polygon[:,0] = np.clip(polygon[:,0], 0, 15)
polygon[:,1] = np.clip(polygon[:,1], 0, 20)
polygon = polygon.reshape(-1).tolist()

In [58]:
polygon = polygon.reshape(-1).tolist()

In [67]:
img = Image.new('L', (width, height), 0)
ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
mask = numpy.array(img).astype(bool)

In [60]:
polygon

[0, 0, 3, 3, 2, 2, 7, 6]

In [64]:
a = np.random.rand(20,15,3)

In [70]:
b = np.array([])

In [76]:
np.vstack([a[mask,:]]).shape

(9, 3)

In [68]:
a[mask,:]

array([[0.91594459, 0.81229897, 0.27488349],
       [0.11908161, 0.93147995, 0.25745658],
       [0.87020372, 0.66832626, 0.5400341 ],
       [0.67830613, 0.56573383, 0.61196238],
       [0.9209504 , 0.5404235 , 0.39998609],
       [0.93988078, 0.32418242, 0.69176026],
       [0.74795211, 0.90799475, 0.16309501],
       [0.31105888, 0.11423364, 0.33865907],
       [0.46094415, 0.49953528, 0.65918019]])

In [77]:
list(range(0, 4, 2))

[0, 2]

In [34]:
a

array([[[1.62254175e-01, 7.44136972e-01, 3.01891005e-01],
        [5.40318615e-01, 3.43615827e-01, 6.38438200e-01],
        [5.03069155e-01, 4.99246613e-01, 1.92547904e-01],
        [6.18874446e-01, 6.04146478e-01, 2.97065762e-01],
        [1.83162008e-01, 4.14596530e-01, 3.20634011e-01],
        [5.50118074e-01, 9.07726350e-01, 5.30130566e-01],
        [4.28358484e-01, 2.71573690e-01, 4.88556584e-01],
        [2.80074321e-01, 7.86202076e-01, 3.21196282e-01],
        [9.29135792e-01, 2.91869093e-01, 4.11074312e-02],
        [3.67521069e-01, 2.66385929e-02, 5.50540274e-01],
        [7.70471730e-01, 9.42992417e-01, 2.60471481e-01],
        [1.07244734e-01, 9.60677018e-01, 7.22168208e-02],
        [1.21598196e-01, 2.39243760e-01, 6.84067602e-01],
        [5.92891708e-01, 8.49056659e-01, 8.86532984e-01],
        [3.35316454e-01, 6.94546412e-01, 5.37854612e-01]],

       [[7.10659650e-01, 7.72538239e-01, 2.95900152e-01],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.0

In [11]:
from model.utility import _gather_feat

ImportError: cannot import name '_gather_feat'

In [15]:
import sys
sys.path.insert(0, "/home/allen/Documents/caffe/python")
sys.path.insert(0, "/home/allen/Documents/boost_1_67_0/stage/lib")
import caffe
from caffe import layers as L
from caffe import params as P

In [13]:
from model.OSNet_PFPN import osnet_x1_0

In [4]:
model = osnet_x1_0(name='osnet', num_classes=13)

In [5]:
caffe_net = caffe.NetSpec()
layer = L.Input(shape=dict(dim=[1, 3, 512, 512]))
caffe_net.tops['data'] = layer
# generate_caffe_prototxt(self, caffe_net, layer)
# print(caffe_net.to_proto())

In [7]:
print(caffe_net.to_proto())

layer {
  name: "data"
  type: "Input"
  top: "data"
  input_param {
    shape {
      dim: 1
      dim: 3
      dim: 512
      dim: 512
    }
  }
}
layer {
  name: "osnet/block0/conv"
  type: "Convolution"
  bottom: "data"
  top: "osnet/block0/conv"
  convolution_param {
    num_output: 64
    bias_term: false
    pad: 3
    kernel_size: 7
    group: 1
    stride: 2
    weight_filler {
      type: "msra"
    }
    dilation: 1
  }
}
layer {
  name: "osnet/block0/conv/bn"
  type: "BatchNorm"
  bottom: "osnet/block0/conv"
  top: "osnet/block0/conv/bn"
}
layer {
  name: "osnet/block0/conv/bn/scale"
  type: "Scale"
  bottom: "osnet/block0/conv/bn"
  top: "osnet/block0/conv/bn"
  scale_param {
    bias_term: true
  }
}
layer {
  name: "osnet/block0/conv/bn/leaky_relu"
  type: "ReLU"
  bottom: "osnet/block0/conv/bn"
  top: "osnet/block0/conv/bn/leaky_relu"
  relu_param {
    negative_slope: 0.009999999776482582
  }
}
layer {
  name: "osnet/block0/reduce/max"
  type: "Pooling"
  bottom: "osne

In [8]:
with open('/home/allen/osnet.prototxt', 'w') as f:
    f.write(str(caffe_net.to_proto()))

In [11]:
net = caffe.Net('/home/allen/osnet.prototxt',caffe.TRAIN)

In [11]:
a = np.ones((1,3,10,10))
a[0,0,...] = 2
a[0,1,...] = 4

In [12]:
net.blobs['data'].data[...] = a

In [7]:
a = torch.rand(1,3,10,10)

In [8]:
b = nn.Upsample(scale_factor=2, mode='bilinear')

In [9]:
b(a).shape

/home/allen/.pyenv/versions/3.6.8/envs/py368/lib/python3.6/site-packages/torch/nn/functional.py:2539: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


torch.Size([1, 3, 20, 20])

In [14]:
net.forward()

{'mul': array([[[[ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.]],
 
         [[16., 16., 16., 16., 16., 16., 16., 16., 16., 16.],
          [16., 16., 16., 16., 16., 16., 16., 16., 16., 16.],
          [16., 16., 16., 16., 16., 16., 16., 16., 16., 16.],
          [16., 16., 16., 16., 16., 16., 16., 16., 16., 16.],
          [16., 16., 16., 16., 16., 16., 16., 16., 16., 16.],
          [16., 16., 16., 16., 16., 16., 16., 16., 16., 16.]

In [13]:
a = 'osnet/block2/0/2a/gate/sigmoid'
a.split('/2a/')

['osnet/block2/0', 'gate/sigmoid']

In [12]:
a[16] = 'b'
a

TypeError: 'str' object does not support item assignment

In [12]:
for name, m in model.named_modules():
    print(m.g_name)

AttributeError: 'OSNet' object has no attribute 'g_name'

In [7]:
c =1
a=b=d=c

In [9]:
b

1

In [2]:
from inplace_abn import InPlaceABN as IABN

In [8]:
from model.OSNet_PFPN import ConvLayer

In [11]:
b = nn.MaxPool2d(3, stride=2, padding=0)

In [12]:
a = torch.rand(1,3,256,256)

In [13]:
b(a).shape

torch.Size([1, 3, 127, 127])

In [11]:
b.copy()

AttributeError: 'ConvLayer' object has no attribute 'copy'

In [16]:
b

ConvLayer(
  (conv): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (1): InPlaceABN(64, eps=1e-05, momentum=0.1, affine=True, activation=leaky_relu[0.01])
    )
  )
)

In [15]:
for n, p in b.named_modules():
    print(n)


conv
conv.0
conv.0.0
conv.0.1


In [11]:
b.activation_param

0.01

In [9]:
isinstance(b, IABN)

True

In [3]:
a = np.zeros((2,4))
b = a[0]
b = 1
a

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [3]:
from app.load_pose import *

/media/allen/mass/deep-learning-works/app/load_pose.py


In [4]:
import os.path as osp

In [5]:
osp.dirname('/media/allen/mass/deep-learning-works/app/load_pose.py')

'/media/allen/mass/deep-learning-works/app'

In [59]:
!pip install colormath

  Running setup.py bdist_wheel for colormath ... done
  Stored in directory: /home/allen/.cache/pip/wheels/20/49/7b/9635bccd6136cc713f623caa85ced89c0c63878f8b2a58fe99
Successfully built colormath
You are using pip version 18.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [60]:
import csv
import numpy as np

from colormath.color_objects import LabColor

# load list of 1000 random colors from the XKCD color chart
# reader = csv.DictReader('lab_matrix.csv)
# lab_matrix = np.array([map(float, row.values()) for row in reader])

# the reference color
color = LabColor(lab_l=69.34,lab_a=-0.88,lab_b=-52.57)

# find the closest match to `color` in `lab_matrix`
delta = color.delta_e_matrix(lab_matrix)
nearest_color = lab_matrix[np.argmin(delta)]

print '%s is closest to %s' % (color, nearest_color)

In [64]:
import pickle
pickle_in = open("/media/allen/mass/lab-matrix.pk","rb")
example_dict = pickle.load(pickle_in)

UnicodeDecodeError: 'ascii' codec can't decode byte 0xea in position 27: ordinal not in range(128)

In [65]:
import pickle
import gzip
import numpy

with open("/media/allen/mass/lab-matrix.pk", 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    p = u.load()
    print(p)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.97913783e-02  1.39075180e-01 -3.78509909e-01]
 [ 1.97913783e-01  1.39075180e+00 -3.78509909e+00]
 ...
 [ 3.99190178e+01 -9.63952677e+00  4.67600222e+01]
 [ 4.02267540e+01 -1.02071577e+01  4.69845878e+01]
 [ 4.05349653e+01 -1.07723151e+01  4.72098975e+01]]


In [67]:
p.shape

(197525, 3)

In [31]:
df = pd.DataFrame(index=['a', 'b'])

In [36]:
df[0]['a'] = 10

In [37]:
df

,0
a,10.0
b,NaN
a,10.0


In [57]:
_df = pd.DataFrame([5], index=['a'])

In [33]:
_df = pd.DataFrame([5], index=['a'])
df = df.append(_df)

In [19]:
_df = pd.DataFrame([50], columns=['b'], index=df.index)
df = df.append(_df)

In [30]:
df[['a', 'b']][0] = pd.Series([10,0])
df.transpose()

,0
a,5
b,50


In [52]:
a = np.zeros((1,10))

In [53]:
d = np.vstack([a,a])

In [54]:
d.shape

(2, 10)

In [56]:
np.hstack([d,d]).shape

(2, 20)

In [4]:
a[:,0] = ''

ValueError: could not convert string to float: 

In [64]:
a = torch.rand(10,5)

In [84]:
a[[2,4,6]][:,[0,1]] = 1

In [2]:
a = np.random.rand(2,5)

In [7]:
a[:,0][:,np.newaxis].shape

(2, 1)

In [12]:
a.mean(axis=0, keepdims=True)


array([[0.72178126, 0.45614343, 0.58079133, 0.5696046 , 0.36692519]])

In [9]:
b

In [65]:
mask = (a[:,0]>0.5)
b = a[mask, 0]
b[0] = 1

In [75]:
np.random.seed(5)

In [79]:
tuple(np.random.randint(0,256,3))

(228, 231, 190)

In [47]:
torch.arange(5)

tensor([0, 1, 2, 3, 4])

In [60]:
import time
start = time.time()

In [61]:
time.time() - start

6.073028802871704

In [67]:
mask

tensor([1, 0, 0, 0, 1, 1, 0, 1, 1, 0], dtype=torch.uint8)

In [68]:
a[~mask]

tensor([[0.4801, 0.5524, 0.8230, 0.6185, 0.4286],
        [0.3869, 0.1417, 0.4957, 0.9334, 0.9559],
        [0.0078, 0.5536, 0.5531, 0.8727, 0.6532],
        [0.1790, 0.5947, 0.2739, 0.8038, 0.6151],
        [0.0363, 0.7932, 0.6058, 0.7874, 0.9235]])

In [310]:
a = torch.Tensor([]).cuda()
b = torch.zeros(2,10)

In [308]:
b[:,1:6] = a

In [309]:
b

tensor([[0.0000e+00, 5.8072e+25, 4.5834e-41, 0.0000e+00, 0.0000e+00, 1.1210e-43,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 2.0179e-43, 0.0000e+00, 5.1483e-02, 4.5835e-41,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]], device='cuda:0')

In [311]:
torch.cat([a,b])

RuntimeError: Expected object of backend CUDA but got backend CPU for sequence element 1 in sequence argument at position #1 'tensors'

In [294]:
mask = a[:,1] > 0.8

In [300]:
a[mask, 2] = 100

In [301]:
a

array([[  0.45377285,   0.48923616,   0.95348065],
       [  0.16525674,   0.80621018, 100.        ]])

In [251]:
a = torch.rand(192,5)
b = torch.rand(3,5)

In [217]:
a[-]

IndexError: index -100 is out of bounds for dimension 0 with size 4

In [176]:
_pdist = lambda A, B: torch.sqrt(A.pow(2).sum(1, keepdim = True) - 2 * torch.mm(A, B.t()) + B.pow(2).sum(1, keepdim = True).t())

In [252]:
dist = _pdist(a,a)
mask = torch.triu(torch.ones_like(dist)) == 0
p_dist = dist[mask.t()]
sorting_idx = torch.argsort(p_dist)
n_similar = 3
similar_idx = sorting_idx[:n_similar]
is_positive = p_dist[similar_idx] > 0.5
pos_idx = similar_idx[is_positive]
neg_idx = similar_idx[~is_positive]

In [253]:
torch.sort(p_dist)[0]

tensor([0.0504, 0.1251, 0.1270,  ..., 1.7210, 1.7736, 1.8300])

In [254]:
pos_idx

tensor([], dtype=torch.int64)

In [271]:
for i in range(len(p_dist)):
    i,j = dist_idx_to_pair_idx(len(a), torch.Tensor([i]))
    dist[i,j]

torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Si

torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Si

torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Si

torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Si

torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Si

...

torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Si

torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Si

torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Si

torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Si

torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Si

In [240]:
def dist_idx_to_pair_idx(d, i):
    """
    :param d: number of samples
    :param i: np.array
    :return:
    """
    if i.size() == 0:
        return None
    b = 1 - 2 * d
    x = torch.floor((-b - torch.sqrt(b ** 2 - 8 * i)) / 2).long()
    y = (i + (x * (b + x + 2) / 2).float() + 1).long()
    return x, y

In [228]:
dist_idx_to_pair_idx(len(a), pos_idx.numpy())

In [227]:
def dist_idx_to_pair_idx(d, i):
    """
    :param d: number of samples
    :param i: np.array
    :return:
    """
    if i.size == 0:
        return None
    b = 1 - 2 * d
    x = np.floor((-b - np.sqrt(b ** 2 - 8 * i)) / 2).astype(int)
    y = (i + x * (b + x + 2) / 2 + 1).astype(int)
    return x, y


In [184]:
torch.triu(dist).t()

tensor([[6.9053e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [7.9396e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.1424e+00, 5.5126e-01,        nan, 0.0000e+00],
        [9.1364e-01, 6.0866e-01, 1.0939e+00, 0.0000e+00]])

In [179]:
pdist(a.numpy())

array([0.79395773, 1.14236806, 0.91363547, 0.55125584, 0.60865651,
       1.09394154])

In [185]:
cdist(a.numpy(), a.numpy())

array([[0.        , 0.79395773, 1.14236806, 0.91363547],
       [0.79395773, 0.        , 0.55125584, 0.60865651],
       [1.14236806, 0.55125584, 0.        , 1.09394154],
       [0.91363547, 0.60865651, 1.09394154, 0.        ]])

In [94]:
a = []
for i, j in zip(range(10), range(10,0,-1)):
    a.append((i,j))

In [98]:
sorted(a, key=lambda x: x[1], reverse=True)

[(0, 10),
 (1, 9),
 (2, 8),
 (3, 7),
 (4, 6),
 (5, 5),
 (6, 4),
 (7, 3),
 (8, 2),
 (9, 1)]

In [132]:
a.squeeze()

tensor(0.3463)

tensor([[0.3281, 0.4009, 0.5728, 0.2792, 0.3339],
        [0.5884, 0.5422, 0.0451, 0.2321, 0.1855],
        [0.1398, 0.5030, 0.3494, 0.1957, 0.3484],
        [0.3795, 0.9957, 0.1088, 0.7144, 0.3690]])

In [129]:
a = torch.rand(1)
b = torch.LongTensor([1,2,0,1])
c = torch.unique(b)

In [125]:
a.is_cuda

False

In [116]:
b = []
for i in range(1, a.shape[0]):
    b.append((a[i-1]-a[i:]).abs().sum(dim=1))

In [121]:
torch.save(torch.cat(b), './b.pt')

In [122]:
torch.load('./b.pt')

tensor([1.1246, 0.6116, 1.5805, 0.9914, 1.3919, 1.5124])

In [103]:
pa = pdist(a.numpy(), metric='minkowski', p=1)
pa

array([1.12463284, 0.61164296, 1.58049339, 0.99141562, 1.39190561,
       1.51243335])

In [86]:
np.triu(cdist(a.numpy(), a.numpy(), metric='minkowski', p=1))

array([[0.        , 1.44685149, 1.41151708, 1.99381679],
       [0.        , 0.        , 1.79237229, 1.24076313],
       [0.        , 0.        , 0.        , 1.80928802],
       [0.        , 0.        , 0.        , 0.        ]])

In [91]:
pa = pdist(a.numpy(), metric='minkowski', p=1)
pos = int(len(pa)*0.5)
np.sort(pa)[-pos]

1.7923722863197327

In [80]:
F.softmax(a, dim=1)[0].sum()

tensor(1.)

In [77]:
a[b == 1].mean(dim=0)

tensor([0.7505, 0.6029, 0.4678, 0.8030, 0.3890])

In [52]:
state = torch.load('/media/allen/mass/deep-learning-works/caffe_models/OSNet_market_cels_triplet_center_Adam_lr_0.00035_warmup_10_0.01_plateau_10_flip_crop_erase_imagenet_0.9495.pth')

In [54]:
for n in state['model'].keys():
    print(n)

backbone.conv1.conv.weight
backbone.conv1.bn.weight
backbone.conv1.bn.bias
backbone.conv1.bn.running_mean
backbone.conv1.bn.running_var
backbone.conv1.bn.num_batches_tracked
backbone.conv2.0.conv1.conv.weight
backbone.conv2.0.conv1.bn.weight
backbone.conv2.0.conv1.bn.bias
backbone.conv2.0.conv1.bn.running_mean
backbone.conv2.0.conv1.bn.running_var
backbone.conv2.0.conv1.bn.num_batches_tracked
backbone.conv2.0.conv2a.conv1.weight
backbone.conv2.0.conv2a.conv2.weight
backbone.conv2.0.conv2a.bn.weight
backbone.conv2.0.conv2a.bn.bias
backbone.conv2.0.conv2a.bn.running_mean
backbone.conv2.0.conv2a.bn.running_var
backbone.conv2.0.conv2a.bn.num_batches_tracked
backbone.conv2.0.conv2b.0.conv1.weight
backbone.conv2.0.conv2b.0.conv2.weight
backbone.conv2.0.conv2b.0.bn.weight
backbone.conv2.0.conv2b.0.bn.bias
backbone.conv2.0.conv2b.0.bn.running_mean
backbone.conv2.0.conv2b.0.bn.running_var
backbone.conv2.0.conv2b.0.bn.num_batches_tracked
backbone.conv2.0.conv2b.1.conv1.weight
backbone.conv2.0.co

In [42]:
from scipy.spatial.distance import cdist, pdist

In [50]:
cdist(a, a)

array([[0.        , 0.64470415, 0.65338649, 0.92443713],
       [0.64470415, 0.        , 0.68928961, 0.81580681],
       [0.65338649, 0.68928961, 0.        , 0.2879812 ],
       [0.92443713, 0.81580681, 0.2879812 , 0.        ]])

In [45]:
a = torch.rand(4,5).numpy()

In [47]:
a

array([[0.29640466, 0.87063336, 0.78799623, 0.5562835 , 0.54426324],
       [0.60057324, 0.9240122 , 0.41293818, 0.14620256, 0.65122455],
       [0.5509925 , 0.5889141 , 0.32295495, 0.73438805, 0.7306729 ],
       [0.7414995 , 0.5126604 , 0.12491226, 0.76451164, 0.7570792 ]],
      dtype=float32)

In [51]:
pdist(a)

array([0.64470415, 0.65338649, 0.92443713, 0.68928961, 0.81580681,
       0.2879812 ])

In [55]:
x = torch.rand(3, 3)
# x[1].fill_(2)
# x[2].fill_(5)
x

tensor([[0.6939, 0.2730, 0.8085],
        [0.4562, 0.7540, 0.0999],
        [0.6742, 0.2814, 0.6902]])

In [68]:
b = F.normalize(x, dim=1)
b.mm(b.t())

tensor([[1.0000, 0.6183, 0.9975],
        [0.6183, 1.0000, 0.6604],
        [0.9975, 0.6604, 1.0000]])

In [66]:
x.renorm(2, 0, 1e-5).mul(1e5).norm(dim=1)

tensor([1.0000, 1.0000, 1.0000])

In [29]:
torch.renorm(x, 2, 0, 1).norm(dim=0)

tensor([1.0916, 0.9455, 0.8067])

In [58]:
model = models.__dict__['mobilenet_v2']()

In [64]:
model.features

Sequential(
  (0): ConvBNReLU(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace)
  )
  (1): InvertedResidual(
    (conv): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace)
      )
      (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (2): InvertedResidual(
    (conv): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace)
      )
      (1): Co

In [65]:
for i in model.features.children():
    print(i)

ConvBNReLU(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU6(inplace)
)
InvertedResidual(
  (conv): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace)
    )
    (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)
InvertedResidual(
  (conv): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace)
    )
    (1): ConvBNReLU(
      (0): Conv2d(96, 96, kernel_size=(3, 3), stride=(2, 2), padd

In [51]:
torch.range(0, 5 - 1).repeat(5, 1).t().contiguous().view(5*5)

/home/allen/.pyenv/versions/3.6.8/envs/py368/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  """Entry point for launching an IPython kernel.


tensor([0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 3., 3., 3.,
        3., 3., 4., 4., 4., 4., 4.])

In [37]:
data = []
for i in range(3):
    data.append(torch.rand(1,3,5))

In [39]:
d = torch.cat(data)

In [47]:
torch.rand(16,3)[:,1].unsqueeze(1).shape

torch.Size([16, 1])

In [16]:
def fliplr(img):
    '''flip horizontal'''
    inv_idx = torch.arange(img.size(3)-1,-1,-1).long().cuda()  # N x C x H x W
    img_flip = img.index_select(3,inv_idx)
    return img_flip

In [31]:
a = torch.rand(3,10)
# a = F.normalize(a)

In [33]:
state = torch.load("/home/allen/osnet_x1_0_imagenet.pth")

In [ ]:
nn.LeakyReLU()

In [36]:
for key in state.keys():
    print(key, state[key].shape)

conv1.conv.weight torch.Size([64, 3, 7, 7])
conv1.bn.weight torch.Size([64])
conv1.bn.bias torch.Size([64])
conv1.bn.running_mean torch.Size([64])
conv1.bn.running_var torch.Size([64])
conv1.bn.num_batches_tracked torch.Size([])
conv2.0.conv1.conv.weight torch.Size([64, 64, 1, 1])
conv2.0.conv1.bn.weight torch.Size([64])
conv2.0.conv1.bn.bias torch.Size([64])
conv2.0.conv1.bn.running_mean torch.Size([64])
conv2.0.conv1.bn.running_var torch.Size([64])
conv2.0.conv1.bn.num_batches_tracked torch.Size([])
conv2.0.conv2a.conv1.weight torch.Size([64, 64, 1, 1])
conv2.0.conv2a.conv2.weight torch.Size([64, 1, 3, 3])
conv2.0.conv2a.bn.weight torch.Size([64])
conv2.0.conv2a.bn.bias torch.Size([64])
conv2.0.conv2a.bn.running_mean torch.Size([64])
conv2.0.conv2a.bn.running_var torch.Size([64])
conv2.0.conv2a.bn.num_batches_tracked torch.Size([])
conv2.0.conv2b.0.conv1.weight torch.Size([64, 64, 1, 1])
conv2.0.conv2b.0.conv2.weight torch.Size([64, 1, 3, 3])
conv2.0.conv2b.0.bn.weight torch.Size([64

In [14]:
a = torch.rand(1,5,5,5).cuda()
a

tensor([[[[3.8353e-01, 5.0457e-01, 3.5444e-01, 3.9653e-01, 1.5101e-01],
          [6.3635e-01, 7.5892e-01, 3.6171e-01, 3.2252e-01, 2.4097e-02],
          [3.7933e-01, 8.5908e-01, 4.7871e-01, 1.7917e-01, 6.2086e-01],
          [4.7553e-01, 8.7144e-03, 1.2487e-02, 8.0865e-01, 5.1826e-01],
          [2.7527e-01, 4.6310e-01, 6.1672e-01, 1.6995e-01, 8.2107e-01]],

         [[2.3942e-01, 7.1076e-01, 5.3386e-02, 1.0184e-01, 8.9193e-01],
          [3.5375e-01, 2.7322e-01, 9.0566e-02, 7.8378e-02, 9.5307e-01],
          [6.3760e-01, 6.2255e-01, 7.2409e-01, 5.0582e-01, 6.7753e-01],
          [4.0158e-01, 6.6376e-01, 7.8078e-01, 5.8723e-01, 1.6339e-01],
          [5.2324e-01, 6.9877e-01, 3.4574e-01, 5.2108e-01, 6.0132e-01]],

         [[5.3149e-01, 2.0126e-01, 5.9462e-01, 4.9545e-01, 1.5206e-01],
          [6.1421e-01, 1.2750e-01, 8.6908e-01, 5.0322e-01, 9.3004e-01],
          [6.5993e-01, 2.9947e-01, 9.8842e-01, 6.6168e-01, 5.8136e-01],
          [5.0136e-01, 9.0975e-01, 4.4190e-01, 7.1898e-01, 9

In [92]:
GAP = nn.AdaptiveAvgPool2d(1)
bn1d = nn.BatchNorm1d(5)
bn2d = nn.BatchNorm2d(5)
a = torch.rand(1,5,5,5)

In [99]:
a = torch.rand(2,5)
b = torch.rand(1,5)

In [101]:
a.min(dim=0)

torch.return_types.min(
values=tensor([0.1507, 0.0842, 0.2267, 0.7682, 0.3895]),
indices=tensor([1, 0, 1, 0, 1]))

In [71]:
from torch.utils.cpp_extension import load
left_pool = load(name='left_pool', sources=['./left_pool.cpp'])

In [72]:
lp = left_pool

In [73]:
lp.forward(a)

[tensor([[[[0.7894, 0.7894, 0.7894, 0.7894, 0.7894, 0.7894, 0.7894, 0.1822],
           [0.9652, 0.9652, 0.9652, 0.9652, 0.9652, 0.9652, 0.6086, 0.1858],
           [0.8070, 0.8070, 0.8070, 0.8070, 0.8070, 0.8070, 0.4677, 0.1252],
           [0.8828, 0.8828, 0.8828, 0.8828, 0.8828, 0.8828, 0.8828, 0.4778]],
 
          [[0.9433, 0.9433, 0.8098, 0.7779, 0.7779, 0.7779, 0.7779, 0.7779],
           [0.9567, 0.9567, 0.8802, 0.8802, 0.8802, 0.4617, 0.4617, 0.1597],
           [0.9540, 0.9540, 0.9540, 0.9540, 0.8848, 0.6371, 0.6371, 0.1503],
           [0.7813, 0.7813, 0.7813, 0.7813, 0.7813, 0.7813, 0.7813, 0.7813]]],
 
 
         [[[0.8743, 0.8743, 0.8031, 0.8031, 0.6313, 0.6313, 0.6313, 0.2811],
           [0.8258, 0.8258, 0.8258, 0.8258, 0.8258, 0.8258, 0.8258, 0.7824],
           [0.7130, 0.7130, 0.7130, 0.7130, 0.4045, 0.4045, 0.4045, 0.1836],
           [0.7058, 0.7058, 0.7058, 0.7058, 0.7058, 0.7058, 0.7058, 0.6405]],
 
          [[0.9490, 0.9490, 0.9490, 0.9490, 0.9490, 0.9490, 0.94

In [82]:
def logger(f):
    def log(*a, **k):
        print(a)
        print(k)
        print(f(*a, **k))
    return log

@logger
def say_hello(msg, show=True):
    return "Hello, " + msg
        

In [84]:
say_hello('wow', show=False)

('wow',)
{'show': False}
Hello, wow


In [13]:
py_left_pool(a)

tensor([[[0, 0, 0, 0],
         [1, 0, 0, 1]],

        [[0, 0, 0, 0],
         [1, 0, 0, 0]]], dtype=torch.uint8)

In [17]:
a.select(3,8-1)[py_left_pool(a)]

tensor([0.7779, 0.7813, 0.9490])

In [44]:
import time

In [46]:
a = a.cpu()

In [47]:
start = time.time()
for i in range(100):
    lp.forward(a)
end = time.time()
print((end-start)/100.0)

0.00011533260345458985


In [172]:
start = time.time()
for i in range(100):
    py_left_pool(a)
end = time.time()
print((end-start)/100.0)

0.0005616402626037598


In [5]:
a = torch.rand(2,2,4,8)
b = torch.rand(2,2,4,8)

In [150]:
a.select(2,3).shape

torch.Size([2, 2, 8])

In [26]:
input_tmp = a.select(2,7)
output_tmp = a.select(2,6)
torch.cat((input_tmp.view(1,-1),output_tmp.view(1,-1)),0).max(0)[0]

tensor([0.2194, 0.8546, 0.3965, 0.7124])

In [42]:
a

tensor([[[[0.7047, 0.7429, 0.5013, 0.6818, 0.2843, 0.4415, 0.1958, 0.0205],
          [0.0220, 0.3194, 0.2110, 0.3296, 0.2507, 0.8083, 0.1025, 0.2461],
          [0.1900, 0.8467, 0.2825, 0.4827, 0.6492, 0.4676, 0.6609, 0.6061],
          [0.4316, 0.9443, 0.3610, 0.8679, 0.7336, 0.4667, 0.8307, 0.6289]]]])

In [43]:
b

tensor([[[[0.1399, 0.4283, 0.2567, 0.9917, 0.3482, 0.1539, 0.6690, 0.0839],
          [0.8300, 0.4666, 0.0359, 0.4574, 0.0788, 0.5924, 0.9688, 0.2396],
          [0.2637, 0.8420, 0.3599, 0.7539, 0.9344, 0.1918, 0.7693, 0.7151],
          [0.7090, 0.6592, 0.6511, 0.0466, 0.3824, 0.7561, 0.9642, 0.9823]]]])

In [143]:
input_ = a
grad_output = b

grad_output = grad_output.clone()

w = input_.size(3)
batch = input_.size(0)

c = input_.clone()
d = grad_output.clone()
for idx in range(1, w):            
    need_copy = c.select(3,w-idx) > input_.select(3,w-idx-1)
    c.select(3,w-idx-1)[need_copy] = c.select(3,w-idx)[need_copy]
    d.select(3,w-idx-1)[need_copy] = d.select(3,w-idx)[need_copy]

In [6]:
def py_left_pool(input_):
    
    w = input_.size(3)
    batch = input_.size(0)

    c = input_.clone()
    for idx in range(1, w):            
        need_copy = c.select(3,w-idx) > input_.select(3,w-idx-1)
        return need_copy
        c.select(3,w-idx-1)[need_copy] = c.select(3,w-idx)[need_copy]
    return c

In [7]:
a.select(3,8-1)[py_left_pool(a)]

tensor([0.7779, 0.7813, 0.9490])

In [8]:
a.select(3,8-1-1)[py_left_pool(a)]

tensor([0.0988, 0.7450, 0.0398])

In [202]:
dd = torch.zeros_like(a.select(3,8-1))
torch.masked_select(a.select(3,8-1), py_left_pool(a), out=dd)

tensor([0.7766, 0.1918, 0.3823, 0.2529, 0.4705, 0.6872])

In [203]:
dd

tensor([0.7766, 0.1918, 0.3823, 0.2529, 0.4705, 0.6872])

In [193]:
py_left_pool(a)

tensor([[[1, 1, 0, 0],
         [1, 0, 0, 0]],

        [[0, 0, 1, 1],
         [0, 1, 0, 0]]], dtype=torch.uint8)

In [152]:
need_copy.shape

torch.Size([2, 2, 4])

In [111]:
idx = 1
c = input_.clone()
d = grad_output.clone()
need_copy = input_.select(3,w-idx) > input_.select(3,w-idx-1)
c.select(3,w-idx-1)[need_copy] = input_.select(3,w-idx)[need_copy]
d.select(3,w-idx-1)[need_copy] = grad_output.select(3,w-idx)[need_copy]

In [144]:
c

tensor([[[[0.9799, 0.9799, 0.9799, 0.9799, 0.9799, 0.9799, 0.7766, 0.7766],
          [0.7283, 0.6894, 0.6894, 0.6894, 0.5106, 0.5106, 0.1918, 0.1918],
          [0.9186, 0.9186, 0.9186, 0.9186, 0.9186, 0.9186, 0.6578, 0.1843],
          [0.9636, 0.9636, 0.8109, 0.8109, 0.8109, 0.2109, 0.2109, 0.1421]],

         [[0.7230, 0.7230, 0.7230, 0.7230, 0.7230, 0.5018, 0.3823, 0.3823],
          [0.9745, 0.9745, 0.9745, 0.9745, 0.9745, 0.9745, 0.7043, 0.4405],
          [0.8437, 0.8437, 0.8437, 0.8437, 0.7172, 0.1205, 0.1205, 0.0592],
          [0.9992, 0.9992, 0.9992, 0.9992, 0.9992, 0.9992, 0.4972, 0.2756]]],


        [[[0.9935, 0.9935, 0.9935, 0.9935, 0.9935, 0.9935, 0.9935, 0.5249],
          [0.9002, 0.7059, 0.7059, 0.5868, 0.5868, 0.5868, 0.4307, 0.2944],
          [0.9255, 0.9255, 0.9255, 0.9255, 0.9255, 0.9255, 0.2529, 0.2529],
          [0.8873, 0.8873, 0.8873, 0.8873, 0.8873, 0.8873, 0.4705, 0.4705]],

         [[0.9719, 0.9719, 0.9719, 0.9719, 0.9719, 0.9719, 0.9719, 0.3358],
    

In [145]:
a

tensor([[[[2.2364e-01, 9.4684e-01, 3.5261e-01, 7.6830e-01, 3.6932e-01,
           9.7990e-01, 3.4830e-02, 7.7655e-01],
          [7.2827e-01, 1.4334e-01, 4.6344e-01, 6.8942e-01, 4.0897e-01,
           5.1061e-01, 6.3116e-02, 1.9181e-01],
          [6.1384e-01, 4.9868e-01, 8.9189e-01, 7.5832e-01, 2.6885e-01,
           9.1857e-01, 6.5777e-01, 1.8426e-01],
          [7.4140e-01, 9.6356e-01, 1.6742e-01, 3.1111e-01, 8.1090e-01,
           1.3007e-01, 2.1089e-01, 1.4210e-01]],

         [[8.2159e-02, 4.7126e-01, 2.9474e-01, 3.6259e-01, 7.2303e-01,
           5.0177e-01, 2.9600e-01, 3.8233e-01],
          [4.7570e-01, 7.2712e-01, 2.8185e-01, 4.5136e-01, 4.2211e-01,
           9.7455e-01, 7.0431e-01, 4.4053e-01],
          [8.3888e-01, 1.5592e-01, 7.0862e-01, 8.4373e-01, 7.1715e-01,
           4.1263e-02, 1.2052e-01, 5.9201e-02],
          [6.1772e-01, 6.0641e-01, 7.4188e-01, 2.8454e-01, 9.9361e-02,
           9.9925e-01, 4.9719e-01, 2.7557e-01]]],


        [[[5.1598e-01, 6.5327e-01, 6.1304e

In [116]:
d

tensor([[[[0.4283, 0.4283, 0.9917, 0.9917, 0.1539, 0.1539, 0.6690, 0.0839],
          [0.4666, 0.4666, 0.4574, 0.4574, 0.5924, 0.5924, 0.2396, 0.2396],
          [0.8420, 0.8420, 0.7539, 0.9344, 0.9344, 0.7693, 0.7693, 0.7151],
          [0.6592, 0.6592, 0.0466, 0.0466, 0.3824, 0.9642, 0.9642, 0.9823]]]])

In [119]:
grad_output

tensor([[[[0.1399, 0.4283, 0.2567, 0.9917, 0.3482, 0.1539, 0.6690, 0.0839],
          [0.8300, 0.4666, 0.0359, 0.4574, 0.0788, 0.5924, 0.9688, 0.2396],
          [0.2637, 0.8420, 0.3599, 0.7539, 0.9344, 0.1918, 0.7693, 0.7151],
          [0.7090, 0.6592, 0.6511, 0.0466, 0.3824, 0.7561, 0.9642, 0.9823]]]])

In [85]:
comp(input_tmp,output_tmp,grad_output_tmp,res_tmp)

a_ tensor([[[0.1958, 0.1025, 0.6609, 0.8307]]])
b_ tensor([[[0.0205, 0.2461, 0.6061, 0.6289]]])
c_ tensor([[[0.1958, 0.1025, 0.6609, 0.8307],
         [0.0205, 0.2461, 0.6061, 0.6289]]])
m tensor([[[0.1958, 0.2461, 0.6609, 0.8307],
         [0.1958, 0.2461, 0.6609, 0.8307]]])
m tensor([[[1., 0., 1., 1.],
         [0., 1., 0., 0.]]])
m1 tensor([[[1., 0.],
         [0., 1.],
         [1., 0.],
         [1., 0.]]])
k tensor([[1., 0., 1., 1.]])
j tensor([[0., 1., 0., 0.]])
z tensor([[1., 1., 1., 1.]])
j tensor([[0., 1., 0., 0.]])
m1 tensor([[[1., 0., 1., 1.],
         [0., 1., 0., 0.]]])
A_ tensor([[[0.6690, 0.9688, 0.7693, 0.9642]]])
B_ tensor([[[0.0839, 0.2396, 0.7151, 0.9823]]])
C_ tensor([[[0.6690, 0.0839],
         [0.9688, 0.2396],
         [0.7693, 0.7151],
         [0.9642, 0.9823]]])
m1 tensor([[[1, 0],
         [0, 1],
         [1, 0],
         [1, 0]]])
res tensor([[[0.6690, 0.2396, 0.7693, 0.9642]]])


In [146]:
input_ = a
grad_output = b
output = torch.zeros_like(input_)

grad_output = grad_output.clone()
res = torch.zeros_like(grad_output)

w = input_.size(3)
batch = input_.size(0)

output_tmp = res.select(3, w-1)
grad_output_tmp = grad_output.select(3, w-1)
output_tmp.copy_(grad_output_tmp)

input_tmp = input_.select(3, w-1)
output.select(3,w-1).copy_(input_tmp)

for idx in range(1, w):

    input_tmp = input_.select(3, w-idx-1)
    output_tmp = output.select(3,w-idx)
    cmp_tmp = torch.cat((input_tmp.view(batch,1,-1),output_tmp.view(batch,1,-1)),1).max(1)[0]
    output.select(3,w-idx-1).copy_(cmp_tmp.view_as(input_tmp))

    grad_output_tmp = grad_output.select(3, w-idx-1)
    res_tmp = res.select(3,w-idx)
    com_tmp = comp(input_tmp,output_tmp,grad_output_tmp,res_tmp)
    res.select(3,w-idx-1).copy_(com_tmp)

In [147]:
c - output

tensor([[[[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.]],

         [[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.]]],


        [[[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.]],

         [[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.]]]])

In [148]:
d - res

tensor([[[[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.]],

         [[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.]]],


        [[[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.]],

         [[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.]]]])

In [126]:
res

tensor([[[[0.4283, 0.4283, 0.9917, 0.9917, 0.1539, 0.1539, 0.6690, 0.0839],
          [0.5924, 0.5924, 0.5924, 0.5924, 0.5924, 0.5924, 0.2396, 0.2396],
          [0.8420, 0.8420, 0.7693, 0.7693, 0.7693, 0.7693, 0.7693, 0.7151],
          [0.6592, 0.6592, 0.0466, 0.0466, 0.9642, 0.9642, 0.9642, 0.9823]]]])

In [122]:
def comp(a,b,A,B):
    batch = a.size(0)
    a_ = a.unsqueeze(1).contiguous().view(batch,1,-1)
    b_ = b.unsqueeze(1).contiguous().view(batch,1,-1)
    c_ = torch.cat((a_,b_),1)
    m = c_.max(1)[0].unsqueeze(1).expand_as(c_)
    m = (c_==m).float()
    m1 = m.permute(0,2,1)
    k = m1[...,0]
    j = m1[...,1]
    z = ((k*j)!=1).float()
    j = z*j
    m1 = torch.cat((k,j),1).unsqueeze(1).view_as(m)

    A_ = A.unsqueeze(1).contiguous().view(batch,1,-1)
    B_ = B.unsqueeze(1).contiguous().view(batch,1,-1)
    C_ = torch.cat((A_,B_),1).permute(0,2,1)
    m1 = m1.long().permute(0,2,1)
    res = C_[m1.long()==1].view_as(a)

    return res

In [84]:
def comp(a,b,A,B):
    batch = a.size(0)
    a_ = a.unsqueeze(1).contiguous().view(batch,1,-1)
    print('a_', a_)
    b_ = b.unsqueeze(1).contiguous().view(batch,1,-1)
    print('b_', b_)
    c_ = torch.cat((a_,b_),1)
    print('c_', c_)
    m = c_.max(1)[0].unsqueeze(1).expand_as(c_)
    print('m', m)
    m = (c_==m).float()
    print('m', m)
    m1 = m.permute(0,2,1)
    print('m1', m1)
    k = m1[...,0]
    print('k', k)
    j = m1[...,1]
    print('j', j)
    z = ((k*j)!=1).float()
    print('z', z)    
    j = z*j
    print('j', j)
    m1 = torch.cat((k,j),1).unsqueeze(1).view_as(m)
    print('m1', m1)
    A_ = A.unsqueeze(1).contiguous().view(batch,1,-1)
    print('A_', A_)
    B_ = B.unsqueeze(1).contiguous().view(batch,1,-1)
    print('B_', B_)
    C_ = torch.cat((A_,B_),1).permute(0,2,1)
    print('C_', C_)
    m1 = m1.long().permute(0,2,1)
    print('m1', m1)
    res = C_[m1.long()==1].view_as(a)
    print('res', res)

In [29]:
com_tmp = comp(input_tmp,output_tmp,grad_output_tmp,res_tmp)
res.select(3,w-idx-1).copy_(com_tmp)

tensor([[[[0.9352, 0.9352, 0.9352, 0.9352, 0.9352, 0.9352, 0.8882, 0.8882],
          [0.4288, 0.4288, 0.4288, 0.4288, 0.2058, 0.2058, 0.1639, 0.0090],
          [0.9080, 0.9080, 0.9080, 0.9080, 0.5551, 0.5551, 0.5551, 0.5551],
          [0.9053, 0.9053, 0.9053, 0.9053, 0.9053, 0.8617, 0.8304, 0.8304]]]])

#### for par accuracy

In [7]:
feat = torch.rand(5,4)
bce = nn.BCEWithLogitsLoss(reduction='none')

In [12]:
feat[0,:].sigmoid()

tensor([0.6057, 0.6746, 0.5272, 0.5545])

In [13]:
feat.sigmoid()

tensor([[0.6057, 0.6746, 0.5272, 0.5545],
        [0.6763, 0.7166, 0.5379, 0.7128],
        [0.5466, 0.5634, 0.6417, 0.7224],
        [0.7129, 0.5966, 0.6445, 0.5050],
        [0.6397, 0.5468, 0.6041, 0.6348]])

In [11]:
feat

tensor([[0.3530, 0.2273, 0.3568, 0.0987],
        [0.8222, 0.6997, 0.4419, 0.2909],
        [0.0654, 0.1974, 0.5089, 0.4087],
        [0.9311, 0.3319, 0.9232, 0.2800],
        [0.0020, 0.7007, 0.2696, 0.8474]])

In [12]:
target = torch.FloatTensor([[1,0,1,-1],[0,1,-1,-1],[1,1,1,-1],[1,-1,0,-1],[-1,-1,1,-1]])
temp_target = torch.zeros_like(target)

In [13]:
target

tensor([[ 1.,  0.,  1., -1.],
        [ 0.,  1., -1., -1.],
        [ 1.,  1.,  1., -1.],
        [ 1., -1.,  0., -1.],
        [-1., -1.,  1., -1.]])

In [14]:
temp_target[target>0] = 1

In [15]:
temp_target

tensor([[1., 0., 1., 0.],
        [0., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.]])

In [16]:
known_target = (target>-1).float()
p = -4.45 * known_target.mean(dim=1).reshape(known_target.size(0),1) + 5.45

In [17]:
known_target

tensor([[1., 1., 1., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 0., 1., 0.],
        [0., 0., 1., 0.]])

In [18]:
p

tensor([[2.1125],
        [3.2250],
        [2.1125],
        [3.2250],
        [4.3375]])

In [110]:
p * known_target

tensor([[2.1125, 2.1125, 2.1125, 0.0000],
        [3.2250, 3.2250, 0.0000, 0.0000],
        [2.1125, 2.1125, 2.1125, 0.0000],
        [3.2250, 0.0000, 3.2250, 0.0000],
        [0.0000, 0.0000, 4.3375, 0.0000]], device='cuda:0')

In [19]:
bce(feat, temp_target)

tensor([[0.5321, 0.8132, 0.5306, 0.7437],
        [1.1865, 0.4033, 0.9383, 0.8491],
        [0.6610, 0.5993, 0.4707, 0.9182],
        [0.3323, 0.8728, 1.2577, 0.8429],
        [0.6942, 1.1037, 0.5674, 1.2040]])

In [112]:
loss_table = p * bce(feat, temp_target)

In [113]:
loss_table[known_target==0] = 0

In [114]:
loss_table

tensor([[0.7921, 1.6502, 0.7409, 0.0000],
        [2.6139, 1.0669, 0.0000, 0.0000],
        [0.9458, 1.3317, 1.2240, 0.0000],
        [1.5182, 0.0000, 2.2886, 0.0000],
        [0.0000, 0.0000, 2.4414, 0.0000]], device='cuda:0')

In [115]:
loss_table.mean()

tensor(0.8307, device='cuda:0')

In [116]:
loss_table[known_target==1].mean()

tensor(1.5103, device='cuda:0')

In [148]:
predicted_target = torch.zeros_like(feat)
predicted_target[feat>=0.5] = 1


In [160]:
predicted_target[known_target==1]

tensor([1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0.], device='cuda:0')

In [173]:
target[known_target==1]

tensor([1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1.], device='cuda:0')

In [174]:
torch.tensor(np.invert(target[known_target==1].cpu().numpy()==1)).long()

tensor([0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0])

In [162]:
predicted_target[known_target==1] * target[known_target==1]

tensor([1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0.], device='cuda:0')

In [211]:
torch.cat([feat,feat], 0)

tensor([[0.0367, 0.1610, 0.5989, 0.5545],
        [0.6178, 0.4195, 0.9821, 0.6457],
        [0.8748, 0.9856, 0.0972, 0.6532],
        [0.9263, 0.8533, 0.1700, 0.6012],
        [0.7935, 0.1132, 0.8305, 0.3816],
        [0.0367, 0.1610, 0.5989, 0.5545],
        [0.6178, 0.4195, 0.9821, 0.6457],
        [0.8748, 0.9856, 0.0972, 0.6532],
        [0.9263, 0.8533, 0.1700, 0.6012],
        [0.7935, 0.1132, 0.8305, 0.3816]])

In [16]:
pt = torch.Tensor(np.load("../par_pt.npy"))
gt = torch.Tensor(np.load("../par_gt.npy"))

In [18]:
pt[0].sigmoid()

tensor([9.5167e-01, 1.0834e-01, 4.7764e-01, 2.0204e-02, 2.1634e-02, 4.0925e-03,
        8.2850e-02, 1.8887e-01, 1.3863e-02, 2.1182e-02, 8.6308e-02, 7.9102e-04,
        9.7025e-01])

#### for loading trained model

In [21]:
state = torch.load("/home/allen/OSNet_market_cels_triplet_center_Adam_lr_0.00035_warmup_10_0.01_plateau_10_flip_crop_erase_imagenet_0.9495.pth")

In [22]:
for n in state['model'].keys():
    print(n)

backbone.conv1.conv.weight
backbone.conv1.bn.weight
backbone.conv1.bn.bias
backbone.conv1.bn.running_mean
backbone.conv1.bn.running_var
backbone.conv1.bn.num_batches_tracked
backbone.conv2.0.conv1.conv.weight
backbone.conv2.0.conv1.bn.weight
backbone.conv2.0.conv1.bn.bias
backbone.conv2.0.conv1.bn.running_mean
backbone.conv2.0.conv1.bn.running_var
backbone.conv2.0.conv1.bn.num_batches_tracked
backbone.conv2.0.conv2a.conv1.weight
backbone.conv2.0.conv2a.conv2.weight
backbone.conv2.0.conv2a.bn.weight
backbone.conv2.0.conv2a.bn.bias
backbone.conv2.0.conv2a.bn.running_mean
backbone.conv2.0.conv2a.bn.running_var
backbone.conv2.0.conv2a.bn.num_batches_tracked
backbone.conv2.0.conv2b.0.conv1.weight
backbone.conv2.0.conv2b.0.conv2.weight
backbone.conv2.0.conv2b.0.bn.weight
backbone.conv2.0.conv2b.0.bn.bias
backbone.conv2.0.conv2b.0.bn.running_mean
backbone.conv2.0.conv2b.0.bn.running_var
backbone.conv2.0.conv2b.0.bn.num_batches_tracked
backbone.conv2.0.conv2b.1.conv1.weight
backbone.conv2.0.co

### numpy

In [1]:
import sys

sys.path.insert(0, '/media/allen/mass/deep-learning-works/')
from tools.eval_par_metrics import eval_par_accuracy

In [5]:
feat = torch.rand(5,4)
target = torch.FloatTensor([[1,0,1,-1],[0,1,-1,-1],[1,1,1,-1],[1,-1,0,-1],[-1,-1,1,-1]])

In [6]:
gt = target.numpy()
predict = feat.numpy()

In [7]:
TPR, FPR, total_precision, attr_TPR, attr_FPR, attr_total_precision = eval_par_accuracy(predict, gt)c

PAR Accuracy: 100%|██████████| 100/100 [00:00<00:00, 3947.99it/s]


In [10]:
TPR

[0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.24999999999921874,
 0.2187499999993164,
 0.2187499999993164,
 0.2187499999993164,
 0.2187499999993164,
 0.2187499999993164,
 0.2187499999993164,
 0.2187499999993164,
 0.2187499999993164,
 0.2187499999993164,
 0.18749999999941405,
 0.18749999999941405,
 0.18749999999941405,
 0.18749999999941405,
 0.18749999999941405,
 0.18749999999941405,
 0.18749999999941405,
 0.15624999999951172,
 0.12499999999960937,
 0.12499999999960937,
 0.12499999999960937,
 0.1249999999996093

In [8]:
attr_TPR

[array([0.09375, 0.0625 , 0.09375, 0.     ], dtype=float32),
 array([0.09375, 0.0625 , 0.09375, 0.     ], dtype=float32),
 array([0.09375, 0.0625 , 0.09375, 0.     ], dtype=float32),
 array([0.09375, 0.0625 , 0.09375, 0.     ], dtype=float32),
 array([0.09375, 0.0625 , 0.09375, 0.     ], dtype=float32),
 array([0.09375, 0.0625 , 0.09375, 0.     ], dtype=float32),
 array([0.09375, 0.0625 , 0.09375, 0.     ], dtype=float32),
 array([0.09375, 0.0625 , 0.09375, 0.     ], dtype=float32),
 array([0.09375, 0.0625 , 0.09375, 0.     ], dtype=float32),
 array([0.09375, 0.0625 , 0.09375, 0.     ], dtype=float32),
 array([0.09375, 0.0625 , 0.09375, 0.     ], dtype=float32),
 array([0.09375, 0.0625 , 0.09375, 0.     ], dtype=float32),
 array([0.09375, 0.0625 , 0.09375, 0.     ], dtype=float32),
 array([0.09375, 0.0625 , 0.09375, 0.     ], dtype=float32),
 array([0.09375, 0.0625 , 0.09375, 0.     ], dtype=float32),
 array([0.09375, 0.0625 , 0.09375, 0.     ], dtype=float32),
 array([0.09375, 0.0625 

In [9]:
total

[0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.6363636363636364,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.6363636363636364,
 0.6363636363636364,
 0.6363636363636364,
 0.6363636363636364,
 0.6363636363636364,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454

In [188]:
gt

array([[ 1.,  0.,  1., -1.],
       [ 0.,  1., -1., -1.],
       [ 1.,  1.,  1., -1.],
       [ 1., -1.,  0., -1.],
       [-1., -1.,  1., -1.]], dtype=float32)

In [28]:
predict

array([[0.36602896, 0.24556452, 0.78518814, 0.83147466],
       [0.6572688 , 0.22959447, 0.55889845, 0.6577302 ],
       [0.04090309, 0.6407464 , 0.02676421, 0.5431648 ],
       [0.12235379, 0.12307644, 0.79519606, 0.56281424],
       [0.2407543 , 0.17260033, 0.3294978 , 0.316217  ]], dtype=float32)

In [27]:
known_gt = gt > -1

In [30]:
predicted_gt = np.zeros_like(predict)
predicted_gt[predict>0.5] = 1
predicted_gt

array([[0., 0., 1., 1.],
       [1., 0., 1., 1.],
       [0., 1., 0., 1.],
       [0., 0., 1., 1.],
       [0., 0., 0., 0.]], dtype=float32)

In [41]:
known_gt[:,0]

array([ True,  True,  True,  True, False])

In [53]:
predicted_gt[:,0][known_gt[:,0]]

array([0., 1., 0., 0.], dtype=float32)

In [52]:
gt[:,0][known_gt[:,0]]

array([1., 0., 1., 1.], dtype=float32)

In [54]:
a = []
for i in range(known_gt.shape[1]):
    a.append((predicted_gt[:,i][known_gt[:,i]] == gt[:,i][known_gt[:,i]]).astype(float).sum())
    

In [55]:
a

[0.0, 2.0, 1.0, 0.0]

In [58]:
known_gt

array([[ True,  True,  True, False],
       [ True,  True, False, False],
       [ True,  True,  True, False],
       [ True, False,  True, False],
       [False, False,  True, False]])

In [64]:
known_gt.sum(axis=0) + 1e-10

array([4.e+00, 3.e+00, 4.e+00, 1.e-10])

In [70]:
np.array(a) / (known_gt.sum(axis=0) + 1e-10)

array([0.        , 0.66666667, 0.25      , 0.        ])

In [71]:
np.array(a).sum() / (known_gt.sum() + 1e-10)

0.2727272727247934

In [43]:
(predicted_gt[known_gt] == gt[known_gt]).astype(float).mean()

0.2727272727272727

In [205]:
(predicted_gt[known_gt] * gt[known_gt]).sum() / gt[known_gt].sum()

0.625

In [207]:
inverse_gt = np.invert(gt[known_gt].astype(bool)).astype(float)
inverse_gt

array([0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0.])

In [210]:
(predicted_gt[known_gt] * inverse_gt).sum() / inverse_gt.sum()

0.3333333333333333

In [209]:
2/3.0

0.6666666666666666

In [157]:
known_target.sum()

tensor(11., device='cuda:0')

In [158]:
(predicted_target[known_target==1] == target[known_target==1]).sum() / known_target.sum()

tensor(0.7273, device='cuda:0')

In [117]:
loss_table.shape[1]

4

In [147]:
loss_table[:,0][known_target[:,0]==1].mean().dim()

0

In [143]:
each_loss = torch.Tensor([loss_table[:,i][known_target[:,i]==1].mean() for i in range(loss_table.shape[1])])

In [146]:
each_loss.dim()

1

In [133]:
each_loss[each_loss != each_loss] = -1

In [139]:
each_loss

tensor([ 1.4675,  1.3496,  1.6737, -1.0000])

In [141]:
isinstance(each_loss, torch.Tensor)
scalar = []
for _tensor in each_loss:
    scalar.append(_tensor.item())

In [142]:
scalar

[1.4674911499023438, 1.3495981693267822, 1.6737191677093506, -1.0]

In [125]:
each_loss = [i for i in each_loss if not torch.isnan(i)]
each_loss

[tensor(1.4675, device='cuda:0'),
 tensor(1.3496, device='cuda:0'),
 tensor(1.6737, device='cuda:0')]

In [18]:
c.mean(dim=1)

tensor([0.7500, 0.5000, 1.0000, 0.7500, 0.5000])

In [70]:
predict = np.load("../par_pt.npy")[:10]
predict = torch.Tensor(predict).sigmoid().numpy()
gt = np.load("../par_gt.npy")[:10]

In [4]:
pt = np.load("../par_pt.npy")
pt = torch.Tensor(pt).sigmoid().numpy()
gt = np.load("../par_gt.npy")

In [5]:
TPR, FPR, total_precision, attr_TPR, attr_FPR, attr_total_precision = eval_par_accuracy(pt, gt)

PAR Accuracy: 100%|██████████| 100/100 [00:04<00:00, 21.02it/s]


In [8]:
attr_TPR[50]

array([0.86469233, 0.6665799 , 0.48633653, 0.637163  , 0.04693786,
       0.9002345 , 0.54111713, 0.66095066, 0.5683787 , 0.6736071 ,
       0.5850184 , 0.8554138 , 0.786637  ], dtype=float32)

In [30]:
known_gt = gt > -1

In [31]:
known_gt

array([[ True, False, False, False, False, False, False, False, False,
        False, False,  True, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False,  True, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False,  True, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False,  True, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False,  True, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False,  True,  True],
       [ True, False, False, False, False, False, False, False, False,
        False, False,  True,  True],
       [ True, False, False, False, False, False, False, False, False,
        False, False,  True, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False,  True, False],
       [ True, False, False,

In [32]:
thresh = 0.5
predicted_gt = np.zeros_like(predict)
predicted_gt[predict >= thresh] = 1

In [43]:
gt

array([[ 1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  0., -1.],
       [ 1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  0., -1.],
       [ 1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  0., -1.],
       [ 1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  0., -1.],
       [ 1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  0., -1.],
       [ 1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  0.,  1.],
       [ 0., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  0.,  1.],
       [ 1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  0., -1.],
       [ 1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  0., -1.],
       [ 1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  0.,  1.]],
      dtype=float32)

In [35]:
predict

array([[9.51669216e-01, 1.08336985e-01, 4.77638811e-01, 2.02043075e-02,
        2.16335524e-02, 4.09247493e-03, 8.28502998e-02, 1.88869074e-01,
        1.38627738e-02, 2.11824831e-02, 8.63084346e-02, 7.91017723e-04,
        9.70245540e-01],
       [8.97413611e-01, 1.46699622e-01, 2.95020133e-01, 1.35976002e-02,
        1.92369539e-02, 1.68039370e-03, 8.03545639e-02, 5.84285080e-01,
        9.78290737e-02, 1.00080697e-02, 4.10405360e-02, 3.05845111e-04,
        9.98244286e-01],
       [8.74959350e-01, 1.53056502e-01, 7.17573389e-02, 4.53980714e-02,
        3.12412549e-02, 3.17711607e-02, 7.04358742e-02, 1.10712647e-01,
        6.63689822e-02, 1.31008420e-02, 1.28084570e-01, 6.15978905e-04,
        9.99514341e-01],
       [8.74959350e-01, 1.53056502e-01, 7.17573389e-02, 4.53980714e-02,
        3.12412549e-02, 3.17711607e-02, 7.04358742e-02, 1.10712647e-01,
        6.63689822e-02, 1.31008420e-02, 1.28084570e-01, 6.15978905e-04,
        9.99514341e-01],
       [6.56111658e-01, 3.69629234e-

In [11]:
known_gt = gt > -1

TPR = []
FPR = []
total_precision = []

attr_TPR = []
attr_FPR = []
attr_total_precision = []

thresh = 0
thresh *= 0.01
predicted_gt = np.zeros_like(pt)
predicted_gt[pt >= thresh] = 1

attr_PP_count = []
attr_TP_count = []
attr_FP_count = []
attr_P_count = []
attr_N_count = []
for i in range(known_gt.shape[1]):
    attr_PP_count.append((predicted_gt[:,i][known_gt[:,i]]).sum())

    attr_TP_count.append((predicted_gt[:,i][known_gt[:,i]] * gt[:,i][known_gt[:,i]]).sum())
    attr_P_count.append(gt[:,i][known_gt[:,i]].sum())

    inverse_gt = np.invert(gt[:,i][known_gt[:,i]].astype(bool)).astype(float)
    attr_FP_count.append((predicted_gt[:,i][known_gt[:,i]] * inverse_gt).sum())
    attr_N_count.append(inverse_gt.sum())

total_precision.append(np.array(attr_TP_count).sum() / (np.array(attr_PP_count).sum() + 1e-10))
TPR.append(np.array(attr_TP_count).sum() / (np.array(attr_P_count).sum() + 1e-10))
FPR.append(np.array(attr_FP_count).sum() / (np.array(attr_N_count).sum() + 1e-10))

attr_total_precision.append(np.array(attr_TP_count) / (np.array(attr_PP_count) + 1e-10))
attr_TPR.append(np.array(attr_TP_count) / (np.array(attr_P_count) + 1e-10))
attr_FPR.append(np.array(attr_FP_count) / (np.array(attr_N_count) + 1e-10))

In [19]:
a = [1,1,1,1,1,1,1]
b = [2,2,2]
c = [3]
count_a, count_b, count_c = len(a), len(b), len(c)

In [23]:
np.random.random(1)

array([0.9692263])

In [110]:
from scipy.spatial.distance import cdist

In [106]:
input_feature_source = np.random.rand(10,5)
input_feature = np.random.rand(8,5)

all_num_source  = input_feature_source.shape[0]
#query_num = probFea.shape[0]
all_num = input_feature.shape[0]    
#feat = np.append(probFea,galFea,axis = 0)
feat = input_feature.astype(np.float16) 

In [113]:
sour_tar_dist = np.power(
        cdist(input_feature, input_feature_source), 2).astype(np.float16)
sour_tar_dist = 1-np.exp(-sour_tar_dist)
source_dist_vec = np.min(sour_tar_dist, axis = 1)
source_dist_vec = source_dist_vec / np.max(source_dist_vec)
source_dist = np.zeros([all_num, all_num])
for i in range(all_num):
    source_dist[i, :] = source_dist_vec + source_dist_vec[i]

In [124]:
source_dist_vec + source_dist_vec[0]

array([0.758 , 0.738 , 1.168 , 0.6963, 1.379 , 0.637 , 0.8853, 0.9414],
      dtype=float16)

In [117]:
source_dist

array([[0.7578125 , 0.73779297, 1.16796875, 0.69628906, 1.37890625,
        0.63720703, 0.88525391, 0.94140625],
       [0.73779297, 0.71777344, 1.14746094, 0.67675781, 1.359375  ,
        0.6171875 , 0.86523438, 0.92138672],
       [1.16796875, 1.14746094, 1.57714844, 1.10644531, 1.7890625 ,
        1.046875  , 1.29492188, 1.3515625 ],
       [0.69628906, 0.67675781, 1.10644531, 0.63525391, 1.31738281,
        0.57617188, 0.82421875, 0.87988281],
       [1.37890625, 1.359375  , 1.7890625 , 1.31738281, 2.        ,
        1.2578125 , 1.50585938, 1.5625    ],
       [0.63720703, 0.6171875 , 1.046875  , 0.57617188, 1.2578125 ,
        0.51660156, 0.76464844, 0.82080078],
       [0.88525391, 0.86523438, 1.29492188, 0.82421875, 1.50585938,
        0.76464844, 1.01269531, 1.06835938],
       [0.94140625, 0.92138672, 1.3515625 , 0.87988281, 1.5625    ,
        0.82080078, 1.06835938, 1.125     ]])

In [120]:
original_dist = cdist(feat,feat).astype(np.float16)  
original_dist = np.power(original_dist,2).astype(np.float16)

In [123]:
source_dist.astype(np.float16)

array([[0.758 , 0.738 , 1.168 , 0.6963, 1.379 , 0.637 , 0.8853, 0.9414],
       [0.738 , 0.718 , 1.147 , 0.677 , 1.359 , 0.617 , 0.865 , 0.9214],
       [1.168 , 1.147 , 1.577 , 1.106 , 1.789 , 1.047 , 1.295 , 1.352 ],
       [0.6963, 0.677 , 1.106 , 0.6353, 1.317 , 0.576 , 0.824 , 0.88  ],
       [1.379 , 1.359 , 1.789 , 1.317 , 2.    , 1.258 , 1.506 , 1.5625],
       [0.637 , 0.617 , 1.047 , 0.576 , 1.258 , 0.5166, 0.7646, 0.821 ],
       [0.8853, 0.865 , 1.295 , 0.824 , 1.506 , 0.7646, 1.013 , 1.068 ],
       [0.9414, 0.9214, 1.352 , 0.88  , 1.5625, 0.821 , 1.068 , 1.125 ]],
      dtype=float16)

In [121]:
original_dist

array([[0.    , 1.304 , 1.01  , 0.5327, 1.    , 1.027 , 0.747 , 0.708 ],
       [1.304 , 0.    , 1.027 , 1.14  , 0.807 , 1.507 , 0.5957, 0.716 ],
       [1.01  , 1.027 , 0.    , 1.685 , 1.127 , 1.424 , 1.351 , 0.0992],
       [0.5327, 1.14  , 1.685 , 0.    , 1.694 , 1.68  , 0.88  , 1.1045],
       [1.    , 0.807 , 1.127 , 1.694 , 0.    , 1.639 , 1.314 , 0.8545],
       [1.027 , 1.507 , 1.424 , 1.68  , 1.639 , 0.    , 0.423 , 1.521 ],
       [0.747 , 0.5957, 1.351 , 0.88  , 1.314 , 0.423 , 0.    , 1.147 ],
       [0.708 , 0.716 , 0.0992, 1.1045, 0.8545, 1.521 , 1.147 , 0.    ]],
      dtype=float16)

In [ ]:
def re_ranking(input_feature_source,input_feature,k1=20,k2=6,lambda_value=0.2, MemorySave = False, Minibatch = 2000, no_rerank=False):

    all_num_source  = input_feature_source.shape[0]
    #query_num = probFea.shape[0]
    all_num = input_feature.shape[0]    
    #feat = np.append(probFea,galFea,axis = 0)
    feat = input_feature.astype(np.float16) 

    print('computing source distance...')
    sour_tar_dist = np.power(
        cdist(input_feature, input_feature_source), 2).astype(np.float16)
    sour_tar_dist = 1-np.exp(-sour_tar_dist)
    source_dist_vec = np.min(sour_tar_dist, axis = 1)
    source_dist_vec = source_dist_vec / np.max(source_dist_vec)
    source_dist = np.zeros([all_num, all_num])
    for i in range(all_num):
        source_dist[i, :] = source_dist_vec + source_dist_vec[i]
    del sour_tar_dist
    del source_dist_vec


    print('computing original distance...')
    if MemorySave:
        original_dist = np.zeros(shape = [all_num,all_num],dtype = np.float16)
        i = 0
        while True:
            it = i + Minibatch
            if it < np.shape(feat)[0]:
                original_dist[i:it,] = np.power(cdist(feat[i:it,],feat),2).astype(np.float16)
            else:
                original_dist[i:,:] = np.power(cdist(feat[i:,],feat),2).astype(np.float16)
                break
            i = it
    else:
        original_dist = cdist(feat,feat).astype(np.float16)  
        original_dist = np.power(original_dist,2).astype(np.float16)
    del feat    
    euclidean_dist = original_dist
    if no_rerank:
        return euclidean_dist, None
    gallery_num = original_dist.shape[0] #gallery_num=all_num
    original_dist = np.transpose(original_dist/np.max(original_dist,axis = 0))
    V = np.zeros_like(original_dist).astype(np.float16)
    initial_rank = np.argsort(original_dist).astype(np.int32)  ## default axis=-1.  

    
    print('starting re_ranking...')
    for i in range(all_num):
        # k-reciprocal neighbors
        forward_k_neigh_index = initial_rank[i,:k1+1]  ## k1+1 because self always ranks first. forward_k_neigh_index.shape=[k1+1].  forward_k_neigh_index[0] == i.
        backward_k_neigh_index = initial_rank[forward_k_neigh_index,:k1+1]  ##backward.shape = [k1+1, k1+1]. For each ele in forward_k_neigh_index, find its rank k1 neighbors
        fi = np.where(backward_k_neigh_index==i)[0]  
        k_reciprocal_index = forward_k_neigh_index[fi]   ## get R(p,k) in the paper
        k_reciprocal_expansion_index = k_reciprocal_index
        for j in range(len(k_reciprocal_index)):
            candidate = k_reciprocal_index[j]
            candidate_forward_k_neigh_index = initial_rank[candidate,:int(np.around(k1/2))+1]
            candidate_backward_k_neigh_index = initial_rank[candidate_forward_k_neigh_index,:int(np.around(k1/2))+1]
            fi_candidate = np.where(candidate_backward_k_neigh_index == candidate)[0]
            candidate_k_reciprocal_index = candidate_forward_k_neigh_index[fi_candidate]
            if len(np.intersect1d(candidate_k_reciprocal_index,k_reciprocal_index))> 2/3*len(candidate_k_reciprocal_index):
                k_reciprocal_expansion_index = np.append(k_reciprocal_expansion_index,candidate_k_reciprocal_index)
            
        k_reciprocal_expansion_index = np.unique(k_reciprocal_expansion_index)  ## element-wise unique
        weight = np.exp(-original_dist[i,k_reciprocal_expansion_index])  
        V[i,k_reciprocal_expansion_index] = weight/np.sum(weight)
    #original_dist = original_dist[:query_num,]    
    if k2 != 1:
        V_qe = np.zeros_like(V,dtype=np.float16)
        for i in range(all_num):
            V_qe[i,:] = np.mean(V[initial_rank[i,:k2],:],axis=0)
        V = V_qe
        del V_qe
    del initial_rank
    invIndex = [] 
    for i in range(gallery_num):
        invIndex.append(np.where(V[:,i] != 0)[0])  #len(invIndex)=all_num
    
    jaccard_dist = np.zeros_like(original_dist,dtype = np.float16)

    
    for i in range(all_num):
        temp_min = np.zeros(shape=[1,gallery_num],dtype=np.float16)
        indNonZero = np.where(V[i,:] != 0)[0]
        indImages = []
        indImages = [invIndex[ind] for ind in indNonZero]
        for j in range(len(indNonZero)):
            temp_min[0,indImages[j]] = temp_min[0,indImages[j]]+ np.minimum(V[i,indNonZero[j]],V[indImages[j],indNonZero[j]])
        jaccard_dist[i] = 1-temp_min/(2-temp_min)

    pos_bool = (jaccard_dist < 0)
    jaccard_dist[pos_bool] = 0.0
    del pos_bool

    # final_dist = jaccard_dist*(1-lambda_value) + original_dist*lambda_value
    final_dist = jaccard_dist*(1-lambda_value) + source_dist*lambda_value
    del original_dist
    del V
    del jaccard_dist
    #final_dist = final_dist[:query_num,query_num:]
    return euclidean_dist, final_dist

In [2]:
feat = torch.rand(4,1)
m = feat.size(0)
distmat = torch.pow(feat, 2).sum(dim=1, keepdim=True).expand(m, m) + \
          torch.pow(feat, 2).sum(dim=1, keepdim=True).expand(m, m).t()
distmat = distmat.addmm(1, -2, feat, feat.t())

In [7]:
np.sqrt(distmat.numpy() + np.eye(m)) * (np.ones_like(distmat) - np.eye(m))

array([[0.        , 0.29118461, 0.09439215, 0.33583853],
       [0.29118461, 0.        , 0.19679237, 0.62702314],
       [0.09439215, 0.19679237, 0.        , 0.43023077],
       [0.33583853, 0.62702314, 0.43023077, 0.        ]])

In [8]:
from scipy.spatial.distance import cdist
cdist(feat.numpy(), feat.numpy())

array([[0.        , 0.2911846 , 0.09439224, 0.33583856],
       [0.2911846 , 0.        , 0.19679236, 0.62702316],
       [0.09439224, 0.19679236, 0.        , 0.4302308 ],
       [0.33583856, 0.62702316, 0.4302308 , 0.        ]])

/home/allen/.pyenv/versions/3.6.8/envs/py368/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in sqrt
  """Entry point for launching an IPython kernel.


array([[          nan, 3.2524011e-01, 5.3368711e-01, 2.6506197e-01],
       [3.2524011e-01,           nan, 2.0844714e-01, 6.0178112e-02],
       [5.3368711e-01, 2.0844714e-01, 1.8126945e-04, 2.6862517e-01],
       [2.6506197e-01, 6.0178112e-02, 2.6862517e-01, 2.6889425e-05]],
      dtype=float32)